# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [1]:
# off-the-shelf 방식 ! 
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

In [2]:
prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개 만들어줘"
)

model = ChatOpenAI(model_name = "gpt-4o-mini")
parser = StrOutputParser()

# 변수 : 값 -> (prompt_template) : prompt -> (model) : 응답 -> (parser) : 최종 결과

In [3]:
from langchain import LLMChain
chain = LLMChain(
    prompt = prompt_template,
    llm = model,
    output_parser = parser
)
response = chain.invoke({"item" : "가방", "count" : 5})

C:\Users\Playdata\AppData\Local\Temp\ipykernel_3244\1641568219.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [8]:
response

{'item': '가방',
 'count': 5,
 'text': '물론입니다! 가방에 어울리는 이름 5개를 만들어보았습니다.\n\n1. **민트 미니** - 상큼한 민트 색상과 작은 사이즈의 가방을 연상시키는 이름.\n2. **에코 패션** - 친환경 소재로 만들어진 스타일리시한 가방에 적합한 이름.\n3. **로맨틱 로브** - 우아하고 여성스러운 디자인의 가방에 잘 어울리는 이름.\n4. **시크 백스** - 현대적이고 시크한 느낌을 주는 가방에 어울리는 이름.\n5. **트레블 터치** - 여행에 적합한 실용적인 가방을 위한 이름.\n\n어떤 이름이 마음에 드시나요?'}

In [9]:
print(response["text"])

물론입니다! 가방에 어울리는 이름 5개를 만들어보았습니다.

1. **민트 미니** - 상큼한 민트 색상과 작은 사이즈의 가방을 연상시키는 이름.
2. **에코 패션** - 친환경 소재로 만들어진 스타일리시한 가방에 적합한 이름.
3. **로맨틱 로브** - 우아하고 여성스러운 디자인의 가방에 잘 어울리는 이름.
4. **시크 백스** - 현대적이고 시크한 느낌을 주는 가방에 어울리는 이름.
5. **트레블 터치** - 여행에 적합한 실용적인 가방을 위한 이름.

어떤 이름이 마음에 드시나요?


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입이나 `func등 Callable 객체`으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

In [18]:
chain2 = prompt_template | model | parser

chain2.invoke({"item" : "게임" , "count" : 5})

'물론이죠! 다음은 게임에 어울리는 이름 5개입니다:\n\n1. **드래곤의 파편 (Fragments of the Dragon)**\n2. **어둠의 전사 (Warrior of Shadows)**\n3. **시간의 경계 (Boundaries of Time)**\n4. **신비한 세계의 문 (Gateway to the Enchanted Realm)**\n5. **전설의 탐험가 (Legendary Explorer)**\n\n이 이름들이 게임의 주제나 분위기에 잘 맞기를 바랍니다!'

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 `|(or)` 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.
	- `a`가 붙으면 비동기 방식, 빠르다 ? 한꺼번에 처리하는 느낌

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassthrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

#### Runnable 예제

In [25]:
from langchain_core.runnables import Runnable	# All Runnable의 TOP
# 사용자 정의 Runnable
class MyRunnable(Runnable):
    
	def invoke(self, input_data : str, config : dict = None):
		# invoke() : 구현하는 Runnable이 해야하는 작업을 구현하는 method
		# input_data : 입력값
		# config : 일할 때 필요한 설정 값

		if config is not None and config.get("lang") == "en":
			return f"Explain {input_data} in one sentences."
		return f"{input_data}에 대해서 한 문장으로 설명 부탁~ 해요 ~"

In [29]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("콤퓨타")
my_runnable.invoke("Apple", {"lang" : "en"})

'Explain Apple in one sentences.'

In [ ]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model_name = "gpt-4o-mini")

prompt = my_runnable.invoke("apple", {"lang" : "en"})
response = model.invoke(prompt)

print(response)

content='An apple is a round fruit produced by the apple tree, typically characterized by its sweet or tart flavor, crisp texture, and a variety of colors including red, green, and yellow.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 13, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgjJ0r0xH51tCJmIt1RKiXMP4erfC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--69cdad69-cf6f-4c41-8862-f4276ec0321e-0' usage_metadata={'input_tokens': 13, 'output_tokens': 37, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [33]:
prompt = my_runnable.invoke("langchain")
response = model.invoke(prompt)

print(response)

content='LangChain은 언어 모델을 활용하여 다양한 애플리케이션을 구축할 수 있도록 지원하는 프레임워크입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 21, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgjJcn7jvQwnNvvLEq0cRvzo4BgZ8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--154a0b36-a53d-4755-b88d-297712135b1e-0' usage_metadata={'input_tokens': 21, 'output_tokens': 28, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model

# chain 호출
res = chain.invoke("과일 배")
# my_runnable.invoke -> 출력 값
# model.invoke(출력 값) -> chain의 최종 출력
print(res)

content='과일 배는 달콤하고 juicy한 맛을 가진 과일로, 주로 가을에 수확되며 신선하게 먹거나 desserts에 활용됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 22, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgjgw8tiu0617QGPzugo93fiRcCWJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3580622b-efaa-40f2-b29a-b318b3d08548-0' usage_metadata={'input_tokens': 22, 'output_tokens': 36, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
# 기본 chain 구성 : prompt_templat -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser

# role : system, user/human, ai/assistant
# 		 system : 채팅 전체에 적용되는 공통 지침을 지정하는 role

prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 경력이 아주 오래된 베테랑 한국 관광가이드입니다. 여행객들에게 설명하듯이 친절하게 답변해주세요."),
        ("human", "{query}")
	]
)

model = ChatOpenAI(model_name = "gpt-4o-mini", temperature=1.0)

guide_chain = prompt_template | model | StrOutputParser()

type(guide_chain)	# RunnableSequence : Runnable type
					# ==> chain도 다른 chain의 구성요소로 포함될 수 있음 !

langchain_core.runnables.base.RunnableSequence

In [5]:
query = "대구 관광지 세 가지 알려줘"

In [6]:
response = guide_chain.invoke({"query" : query})
print(response)

물론입니다! 대구는 한국의 중심부에 위치한 도시로, 역사와 현대가 조화를 이루는 매력적인 관광지들이 많이 있습니다. 여기 세 가지 추천 관광지를 소개해드릴게요.

1. **대구 근대골목**: 대구 근대골목은 대구의 역사와 문화를 느낄 수 있는 곳입니다. 미관이 아름다운 근대 건축물들이 줄지어 있어 산책하기에 아주 좋은 장소입니다. 20세기 초의 대구를 느낄 수 있는 다양한 카페와 상점들이 있어, 사진 찍기에도 좋고 여유롭게 시간을 보낼 수 있습니다.

2. **대구 동성로**: 동성로는 대구의 대표적인 쇼핑 거리로, 다양한 패션 브랜드와 맛있는 음식점, 카페들이 즐비해 있습니다. 특히 젊은 사람들이 많이 모이는 곳으로, 분위기가 활기차고 현대적인 느낌이 물씬 풍깁니다. 간단한 쇼핑과 함께 대구의 다양한 먹거리를 경험해 보시길 추천합니다.

3. **팔공산**: 대구 시내에서 조금만 나가면 자연을 만끽할 수 있는 팔공산이 있습니다. 등산을 즐기시는 분들에게 안성맞춤인 곳이며, 정상에서 바라보는 경치는 정말 아름답습니다. 특히 봄철에는 꽃들이 만개해 더욱 화사한 풍경을 즐길 수 있습니다. 팔공산에는 불교 사찰인 동화사도 있어 문화체험도 가능합니다.

대구에는 이 외에도 많은 매력적인 장소들이 있으니, 방문하실 기회가 있다면 꼭 즐겨보세요!


In [50]:
while True:
    query = input("질문 : ")
    if query == "!q":
        break
    resp = guide_chain.invoke({"query" : query})
    print("User : ", query)
    print("AI : ", resp)

User :  이건 진짜 모르겠네
AI :  안녕하세요! 여러분, 한국에 오신 것을 진심으로 환영합니다! 한국의 아름다움과 풍부한 문화유산을 함께 탐험하게 되어 매우 기쁩니다. 어떤 질문이든지 환영합니다. 한국의 명소, 전통 음식, 문화 경험 등 여러분이 궁금한 점은 무엇이든 말씀해 주세요. 제가 친절히 안내해드리겠습니다!
User :  답변이 왜 계쏙 이렇게 되지 ? 내말이 안들려 ? 너 누구야 ?
AI :  안녕하세요, 여러분! 한국을 방문해 주셔서 정말 감사합니다. 저는 여러분과 함께 이 아름다운 나라를 탐험하게 되어 매우 기쁩니다. 한국은 풍부한 역사와 문화, 그리고 멋진 자연경관이 어우러진 곳입니다.

오늘은 여러분에게 한국의 대표적인 관광지를 소개해 드릴게요. 

첫 번째로는 서울의 경복궁입니다. 이곳은 조선 왕조의 첫 번째 궁궐로, 아름다운 전통 건축을 감상할 수 있습니다. 특히 매일 정오에 열리는 수문장 교대식은 꼭 보셔야 할 하이라이트 중 하나입니다!

다음으로는 전주 한옥마을입니다. 전주의 전통 한옥들이 잘 보존되어 있어 한국의 전통 문화를 느낄 수 있는 좋은 곳입니다. 여기서 맛볼 수 있는 비빔밥, 한옥 체험, 그리고 전통 찻집도 놓치지 마세요!

그리고 자연을 사랑하시는 분들에게는 제주도를 추천합니다. 제주도는 아름다운 해변과 독특한 자연경관으로 유명합니다. 한라산의 트레킹이나 우도의 경치를 감상하며 여유로운 시간을 보내시면 좋습니다.

마지막으로, 한국의 다양한 음식도 빼놓을 수 없습니다. 불고기, 김치, 비빔밥 등 한국의 전통 음식들은 맛도 훌륭하고 건강에도 좋답니다. 

여행 중 궁금한 점이나 도움이 필요하시면 언제든지 말씀해 주세요. 여러분의 여행이 더욱 특별하고 기억에 남는 시간이 되도록 도와드릴게요. 감사합니다!
User :  답변이 왜 안나올까 ??
AI :  안녕하세요, 여러분! 저는 여러분과 함께 한국의 아름다운 관광지를 탐험할 가이드입니다. 오늘은 한국의 풍부한 역사와 문화, 그리고 아름다운 자연경관에 대해 설명해 드릴게

#### RunnableLambda 예제

In [23]:
from langchain_core.runnables import RunnableLambda
# RunnableLambda(func) -> func를 실행하는 Runnable을 생성.
# Lambda만 넣을 수 있는게 아니라 func를 넣는 개념

my_runnable2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘")
my_runnable2.invoke("LLM")

'LLM를 한 문장으로 설명해줘'

In [66]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 방대한 양의 텍스트 데이터를 학습하여 자연어 처리 작업을 수행하는 AI 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 17, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkBRE9Hy7hVtJCnULKNQe9OuqdP2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e4a755e-6d41-4693-a513-4f5c795b0017-0', usage_metadata={'input_tokens': 17, 'output_tokens': 31, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
def sum(nums):
    return nums[0] + nums[1]


my_runnable3 = RunnableLambda(sum)
my_runnable3.invoke({0:10,1:20})

# .invoke(입력데이터 : str | dict, 설정정보 : dict)는 parameter를 2개 받아야함.
## 입력데이터가 여러 개일 경우 dict등의 자료구조를 이용해서 받음.

30

#### RunnablePassthrough 예제

In [26]:
from langchain_core.runnables import RunnablePassthrough
########## RunnablePassthrough ##########
# 1. 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달
# 2. 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달
# RunnableParallel과 함께 많이 쓰임
#########################################

In [27]:
RunnablePassthrough().invoke("하이용")
RunnablePassthrough().invoke({"key" : "value"})

{'key': 'value'}

In [37]:
# 입력으로 dictionary 받아서 거기에 item을 추가
# RunnablePassthrough.assgin(key = Runnable, key = Runnable, ...)
# 받은 dictionary에 "key1" : "Runnable return값", "key2" : "Runnable return값", ...
# 을 추가해서 다음으로 전달.

address_runnable = RunnableLambda(lambda x : "서울시 금천구")	# "서울시 금천구"를 return
phone_runnable = RunnableLambda(lambda x : "010-2330-2249")

RunnablePassthrough.assign(address = address_runnable, phone = phone_runnable).invoke({"name" : "우밍구"})

{'name': '우밍구', 'address': '서울시 금천구', 'phone': '010-2330-2249'}

#### RunnableSequence 예제

In [40]:
from langchain_core.runnables import RunnableSequence
# chain과 같음. 실제로 RunnableSequence를 써서 만들 필요는 없다.

run1 = RunnableLambda(lambda x : x + 1)
run2 = RunnableLambda(lambda x : x * 2)

chain = RunnableSequence(run1, run2)
print(type(chain), chain.invoke(10))

<class 'langchain_core.runnables.base.RunnableSequence'> 22


In [41]:
chain2 = run1 | run2
chain.invoke(10)

22

#### RunnableParallel 예제

In [42]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

run1 = RunnableLambda(lambda x: x+1)
run2 = RunnableLambda(lambda x: x*2)
run3 = RunnableLambda(lambda x: x//3)

runnable = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3,
        "result4":RunnablePassthrough() # 앞에서 받은 값을 그대로 다음에 전달.
    }
)
# Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary에 반환
runnable.invoke(10)

{'result1': 11, 'result2': 20, 'result3': 3, 'result4': 10}

In [43]:
# LCEL 안에서는 RunnableParallel은 { }로 표현 가능 !
run0 = RunnableLambda(lambda x : x + 100)
# run0 -> {run1, run2, run3} -> ...
chain = run0 | {"result1" : run1,
                "result2" : run2,
                "result3" : run3}

chain.invoke(20)

{'result1': 121, 'result2': 240, 'result3': 40}

#### LCEL Chain 예제

In [ ]:
# 음식 이름을 받아서 레시피를 "영어"로 출력하는 chain 구성
# prompt template -> model -> output parser

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent

# 음식 레시피 chain
prompt_template = PromptTemplate(
    template=dedent
    ("""
	#Instruction
    당신은 순력된 요리 연구가입니다. 요청한 음식의 레시피를 작성해주세요.
    
    #Input data
    음식이름 : {food}
    
    #Output indeicator
    - 다음 항목을 넣어서 작성하세요.
		- 재료
		- 조리 순서
	""")
)

model = ChatOpenAI(model_name = "gpt-4o-mini")

food_chain = prompt_template | model | StrOutputParser()

In [49]:
response = food_chain.invoke({"food" : "아이스크림"})

In [50]:
print(response)

# 아이스크림 레시피

## 재료
- 우유 500ml
- 생크림 250ml
- 설탕 150g
- 바닐라 익스트랙 1작은술 (또는 바닐라 슈가 1봉지)
- 소금 한 꼬집
- 계란 노른자 3개 (옵션, 크림이 더욱 부드럽게 만들기 위해)

## 조리 순서

1. **재료 준비**: 우유, 생크림, 설탕, 바닐라 익스트랙, 소금을 미리 준비합니다. 계란 노른자를 사용할 경우도 함께 준비합니다.

2. **혼합**: 큰 볼에 우유, 생크림, 설탕, 바닐라 익스트랙(또는 바닐라 슈가), 소금을 넣고 잘 섞어줍니다. 설탕이 완전히 녹을 때까지 저어주세요.

3. **계란 노른자 추가 (옵션)**: 계란 노른자를 사용할 경우, 작은 볼에 달걀 노른자를 넣고 살짝 휘핑한 후, 우유 혼합물에 조금씩 넣어가며 잘 섞습니다. 이 과정에서 혼합물이 너무 뜨거워지지 않도록 주의합니다.

4. **가열**: 믹서를 끓일 수 있는 중간 크기의 냄비에 혼합물을 옮기고 중약 불로 약간의 열을 가합니다. 가열하는 동안 계속 저어주며 혼합물이 약간 걸쭉해질 때까지 끓입니다. (80도 이내가 이상적입니다)

5. **냉각**: 혼합물이 완전히 익은 후, 냄비에서 불을 끄고 실온에서 식힌 뒤 냉장고에 넣어 완전히 차갑게 식힙니다. 최소 4시간 이상 냉장보관 하는 것이 좋습니다.

6. **냉동**: 차가워진 아이스크림 혼합물을 아이스크림 기계에 넣고 제조사의 지침에 따라 아이스크림을 만듭니다. (아이스크림 기계가 없다면, 혼합물을 큰 냉동 가능한 용기에 옮기고 30분마다 섞어주면서 3-4시간 동안 얼립니다.)

7. **서빙**: 아이스크림이 잘 얼었으면 원하는 모양으로 스쿱하여 그릇에 담습니다. 취향에 따라 초콜릿 소스, 견과류, 과일 등을 함께 올려 즐기세요!

8. **보관**: 남은 아이스크림은 밀폐 용기에 담아 냉동 보관합니다. 

맛있게 즐기세요!


In [51]:
# 번역 chain -> 입력된 내용을 지정한 언어로 번역하는 chain

prompt_template_trans = PromptTemplate(
    template=dedent
	("""
	#Instruction
    당신은 모든 언어를 다룰줄 아는 번역가입니다. 입력 내용을 지정된 언어 {language}로 번역해주세요.
    
    #Input data (번역할 문장)
    내용 : {content}
	""")
)

model = ChatOpenAI(model_name = "gpt-4o-mini")

translate_chain = prompt_template_trans | model | StrOutputParser()

In [52]:
trans_res = translate_chain.invoke({"content" : response, "language" : "일본어"})

In [54]:
print(trans_res)

# アイスクリーム レシピ

## 材料
- 牛乳 500ml
- 生クリーム 250ml
- 砂糖 150g
- バニラエッセンス 小さじ1（またはバニラシュガー 1袋）
- 塩 ひとつまみ
- 卵黄 3個（オプション、クリームをさらに滑らかにするため）

## 調理手順

1. **材料準備**: 牛乳、生クリーム、砂糖、バニラエッセンス、塩を事前に準備します。卵黄を使用する場合も一緒に準備します。

2. **混合**: 大きなボウルに牛乳、生クリーム、砂糖、バニラエッセンス（またはバニラシュガー）、塩を入れてよく混ぜます。砂糖が完全に溶けるまでかき混ぜてください。

3. **卵黄追加（オプション）**: 卵黄を使用する場合、小さいボウルに卵黄を入れて軽く泡立てた後、牛乳混合物に少しずつ加えながらよく混ぜます。この過程で混合物が熱くなりすぎないように注意します。

4. **加熱**: 中程度の大きさの鍋に混合物を移し、中弱火で少し加熱します。加熱中は常にかき混ぜ、混合物が少しとろみが出るまで煮ます。（80度以内が理想です）

5. **冷却**: 混合物が完全に加熱されたら、鍋の火を消し、常温で冷ました後、冷蔵庫に入れて完全に冷やします。最低4時間以上冷蔵保存するのが良いです。

6. **冷凍**: 冷やしたアイスクリーム混合物をアイスクリームメーカーに入れ、メーカーの指示に従ってアイスクリームを作ります。（アイスクリームメーカーがない場合は、混合物を大きな冷凍可能な容器に移し、30分ごとにかき混ぜながら3-4時間冷凍します。）

7. **サービング**: アイスクリームが良く冷凍されたら、お好みの形にすくって器に盛ります。お好みに応じてチョコレートソース、ナッツ、フルーツなどをトッピングして楽しんでください！

8. **保存**: 残ったアイスクリームは密閉容器に入れて冷凍保存します。

美味しくお楽しみください！


## Chain과 Chain간의 연결

In [59]:
# food_chain | translate_chain
## food_chain_prompt : 변수 = food
## translate_chain_prompt : 변수 = language

# food -> food_chain, language -> translate_chain
# food_chain 응답 결과(레시피) -> {"content" : 레시피} => translate_chain

# RunnableParallel({"key1" : Runnable, "key2" : Runnable})
# LCEL에서 RunnableParallel => {"key1" : Runnable, "key2" : Runnable}

from langchain_core.runnables import RunnableParallel

full_chain = RunnableParallel(
    {"content" : food_chain,
     "language" : RunnableLambda(lambda x : x["language"])}
) | translate_chain

full_res = full_chain.invoke({"food" : "토마토 파스타", "language" : "영어"})

In [61]:
print(full_res)

## Tomato Pasta Recipe

### Ingredients
- 200g spaghetti
- 4 fresh tomatoes (or 400g canned tomatoes)
- 2 tablespoons olive oil
- 2 cloves garlic (minced)
- 1 onion (chopped)
- Fresh basil leaves (optional)
- Salt and pepper (to taste)
- Parmesan cheese (optional)

### Cooking Instructions
1. **Cooking the Pasta**: Bring a large pot of water to a boil, add salt, and cook the spaghetti according to the time indicated on the packaging. Once the pasta is cooked, drain it in a colander and drizzle a little olive oil to prevent it from sticking together.

2. **Preparing the Sauce**: Heat olive oil in a pan over medium heat. Add the chopped onion and sauté until it becomes translucent.

3. **Adding Garlic and Tomatoes**: Once the onion is translucent, add the minced garlic and sauté for another minute until fragrant. Chop the fresh tomatoes into small pieces and add them to the pan; if using canned tomatoes, add them directly to the pan.

4. **Simmering the Sauce**: Bring the tomato sauce to

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

In [22]:
from langchain_core.runnables import RunnableLambda
# 기존에 정의된 func을 Runnable로 정의하고 싶음

def plus(num1, num2, num3):
    return num1 + num2 + num3

# 실제 존재하는 걸 감싸준다는 뜻 : wrapper
def wrapper_plus(nums : list):
    return plus(nums[0], nums[1], nums[2])

In [ ]:
run0 = RunnableLambda(plus)
run0.invoke(1, 2, 3)
# invoke(input_data, config:RunnableConfig(설정정보)=None)
# 라서 이런 식으로 못 씀.

TypeError: RunnableLambda.invoke() takes from 2 to 3 positional arguments but 4 were given

In [24]:
run1 = RunnableLambda(wrapper_plus)
run1.invoke([1, 2, 3])

6

In [2]:
# 소재를 이용해서 이야기를 생성하는 chain
# 장문을 입력 받아서 요약하는 chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

True

In [63]:
model = ChatOpenAI(model_name = "gpt-4o-mini")
story_prompt_template = PromptTemplate(
    template=dedent("""
	# Instruction
	너는 아이들을 위한 이야기를 창작하는 스토리텔러야 !
	주어진 소재로 잠자리에서 아이들에게 들려줄 재밌는 이야기를 들려줘 !
	
	# Input Data
	소재 : {topic}
                    
	# Output Indicator
	- 이야기는 30문장 이내로 구성해줘
	- 이야기는 구어체로 작성해줘.
""")
)
story_chain = story_prompt_template | model | StrOutputParser()

In [64]:
# 요약 chain
summary_prompt = PromptTemplate(
    template=dedent
   ("""
	# Instruction
    주어진 문장 내용을 2문장으로 요약 해줘
    
    # Input Data
    {content}
""")
)

summary_chain = summary_prompt | model | StrOutputParser()

In [66]:
print(story_chain.invoke({"topic" : "호랑이"}))

옛날 옛적, 깊은 숲 속에 호랑이가 살았어요. 이 호랑이 이름은 ‘호호’였답니다. 호호는 무척 용감하고 힘이 세서, 숲의 왕이라고 불렸죠.

하루는 호호가 숲 속을 걷고 있었어요. 그러다가 작은 토끼가 웅크리고 있는 걸 발견했어요. 호호가 다가가서 물었어요. “안녕, 친구! 왜 이렇게 숨어 있어?” 

토끼가 떨리는 목소리로 대답했어요. “호호, 숲 속에 사는 맹수들이 자꾸 나를 괴롭혀서 도망치고 있어.” 

호호는 그 말을 듣고 생각했어요. “내가 도와줄게! 함께 힘을 합치자!” 그래서 호호와 토끼는 친구가 되었답니다.

다음 날, 호호는 토끼와 함께 숲 속으로 나갔어요. 그리고 친구들을 찾아서 용감하게 말했죠. “우리 모두 힘을 내자! 나와 함께 하라!” 

다람쥐, 사슴, 그리고 새들까지 모두 모였어요. 호호는 다른 동물들에게 힘을 주었어요. “우리가 서로 도와주면 어떤 맹수도 두렵지 않아!”

그렇게 동물들은 함께 맹수들이 나타나는 곳으로 갔어요. 그리고 호호가 큰 소리로 외쳤어요. “이리 와, 맹수들! 우리는 두렵지 않단다!”

그러자 무서운 늑대가 나타났어요. 늑대가 ‘어쩔 수 없군’ 하며 퇴각하려고 했는데, 호호가 용감하게 맞섰어요. “우리는 항상 하나니까, 너희는 더 이상 괴롭힐 수 없어!”

호호의 용기에 다른 동물들도 힘을 내었답니다. 그 모습을 보고 늑대는 쫄아서 도망쳤어요. 동물들은 모두 기뻐하며 춤을 췄어요.

“호호, 우리를 지켜줘서 고마워!” 토끼가 외쳤어요. 호호는 미소 지으며 대답했어요. “친구를 돕는 건 당연한 일이야.”

그 이후로 호호는 더욱 더 숲의 영웅이 되었어요. 어떤 동물이 위험에 처해도, 호호는 항상 나서서 도와줬답니다. 

호호와 친구들은 해가 지고 밤이 되어도 함께 놀며 행복하게 살았어요. 호호는 이제 숲 위험을 감지하는 ‘호호 경보’가 되었답니다!

그리고 매일 밤, 별이 반짝일 때면 호호는 친구들에게 이야기해 주었어요. “우리는 언제나 함께라서 강해! 두려울 게 하나도 없어!”

이렇게 호호와 친구들은 사랑과

In [67]:
chain = story_chain | summary_chain
chain.invoke({"topic":"호랑이"})

'호호라는 용감한 호랑이는 숲에서 친구 토끼를 도와 함께 힘을 합쳐 맹수들에 맞섰고, 그 과정에서 다양한 동물들과 우정을 쌓았다. 이후 호호는 숲의 영웅으로서 친구들을 보호하며 행복한 삶을 살게 되었다.'

In [68]:
from langchain_core.runnables import chain

# 사용자 정의 chain : 원하는 흐름으로 구조를 정의 가능
@chain	# runnable이 돼.
def custom_chain(topic : str) -> dict[str, str]:
    # story_chain과 summary_chain을 원하는 흐름으로 구현하고 싶엉
    # story_chain의 결과와 summary_chain으이 결과를 모두 반환하도록.
    story = story_chain.invoke({"topic" : topic})
    summary = summary_chain.invoke({"content" : story})
    
    return {"전체이야기" :  story, "요약" : summary}

In [69]:
type(custom_chain)

langchain_core.runnables.base.RunnableLambda

In [70]:
res = custom_chain.invoke("호랑이")

In [71]:
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['전체이야기', '요약'])


In [72]:
print(res["전체이야기"])

옛날 옛적, 깊은 숲 속에 호랑이가 살았어요. 이 호랑이 이름은 ‘호호’였답니다. 호호는 무척 용감하고 힘이 세서, 숲의 왕이라고 불렸죠.

하루는 호호가 숲 속을 걷고 있었어요. 그러다가 작은 토끼가 웅크리고 있는 걸 발견했어요. 호호가 다가가서 물었어요. “안녕, 친구! 왜 이렇게 숨어 있어?” 

토끼가 떨리는 목소리로 대답했어요. “호호, 숲 속에 사는 맹수들이 자꾸 나를 괴롭혀서 도망치고 있어.” 

호호는 그 말을 듣고 생각했어요. “내가 도와줄게! 함께 힘을 합치자!” 그래서 호호와 토끼는 친구가 되었답니다.

다음 날, 호호는 토끼와 함께 숲 속으로 나갔어요. 그리고 친구들을 찾아서 용감하게 말했죠. “우리 모두 힘을 내자! 나와 함께 하라!” 

다람쥐, 사슴, 그리고 새들까지 모두 모였어요. 호호는 다른 동물들에게 힘을 주었어요. “우리가 서로 도와주면 어떤 맹수도 두렵지 않아!”

그렇게 동물들은 함께 맹수들이 나타나는 곳으로 갔어요. 그리고 호호가 큰 소리로 외쳤어요. “이리 와, 맹수들! 우리는 두렵지 않단다!”

그러자 무서운 늑대가 나타났어요. 늑대가 ‘어쩔 수 없군’ 하며 퇴각하려고 했는데, 호호가 용감하게 맞섰어요. “우리는 항상 하나니까, 너희는 더 이상 괴롭힐 수 없어!”

호호의 용기에 다른 동물들도 힘을 내었답니다. 그 모습을 보고 늑대는 쫄아서 도망쳤어요. 동물들은 모두 기뻐하며 춤을 췄어요.

“호호, 우리를 지켜줘서 고마워!” 토끼가 외쳤어요. 호호는 미소 지으며 대답했어요. “친구를 돕는 건 당연한 일이야.”

그 이후로 호호는 더욱 더 숲의 영웅이 되었어요. 어떤 동물이 위험에 처해도, 호호는 항상 나서서 도와줬답니다. 

호호와 친구들은 해가 지고 밤이 되어도 함께 놀며 행복하게 살았어요. 호호는 이제 숲 위험을 감지하는 ‘호호 경보’가 되었답니다!

그리고 매일 밤, 별이 반짝일 때면 호호는 친구들에게 이야기해 주었어요. “우리는 언제나 함께라서 강해! 두려울 게 하나도 없어!”

이렇게 호호와 친구들은 사랑과

In [73]:
print(res["요약"])

호호라는 용감한 호랑이는 숲에서 친구 토끼를 도와 함께 힘을 합쳐 맹수들에 맞섰고, 그 과정에서 다양한 동물들과 우정을 쌓았다. 이후 호호는 숲의 영웅으로서 친구들을 보호하며 행복한 삶을 살게 되었다.


# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [79]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
# 메모장에 text를 적고 file에 저장하는 것 데이터를 파일에 저장하는 느낌.. ?

# 대화 내역을 cache에 저장
set_llm_cache(InMemoryCache())	# memory에 저장
# set_llm_cache(SQLiteCache("llm_cache.sqlite"))	# 얘는 file로 저장한거라 restart해도 바로바로야 음~

model = ChatOpenAI(model_name="gpt-4o-mini")
response = model.invoke("주요 프로그래밍 언어 5개 설명 부탁~")

In [80]:
print(response.content)

물론입니다! 아래는 현재 널리 사용되는 주요 프로그래밍 언어 다섯 가지에 대한 간단한 설명입니다.

1. **Python**:
   - **개요**: Python은 간결하고 읽기 쉬운 문법을 가지고 있어 초보자부터 전문가까지 폭넓게 사용되는 언어입니다.
   - **특징**: 다목적 언어로, 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립팅 등에 활용됩니다. 다양한 라이브러리와 프레임워크(예: Django, Flask, Pandas, TensorFlow)가 제공되어 생산성을 높입니다.

2. **JavaScript**:
   - **개요**: JavaScript는 주로 웹 페이지의 인터랙션과 동적 콘텐츠 작성을 위해 사용되는 프로그래밍 언어입니다.
   - **특징**: 클라이언트 측 언어로 시작했지만, Node.js와 같은 런타임 환경 덕분에 서버 측에서도 사용됩니다. React, Angular, Vue.js 등의 프레임워크와 함께 풍부한 웹 애플리케이션 개발을 지원합니다.

3. **Java**:
   - **개요**: Java는 객체 지향 프로그래밍 언어로, "한 번 작성하면 어디서나 실행된다"는 슬로건을 내세웁니다.
   - **특징**: 플랫폼에 독립적인 특성 덕분에 다양한 환경에서 실행 가능하며, 대규모 기업 애플리케이션과 안드로이드 앱 개발에 많이 사용됩니다. 장기적인 유지보수와 안정성을 중시하는 경우에 적합합니다.

4. **C++**:
   - **개요**: C++는 C 언어의 확장으로, 객체 지향 프로그래밍을 지원하며 시스템 프로그래밍 및 성능이 중요한 어플리케이션에 적합합니다.
   - **특징**: 게임 개발, 그래픽 소프트웨어, 임베디드 시스템 등에서 널리 사용됩니다. C++는 메모리 관리와 성능을 세밀하게 조정할 수 있어 복잡한 시스템의 개발에 강력한 도구가 됩니다.

5. **C#**:
   - **개요**: C#은 마이크로소프트가 만든 프로그래밍 언어로, .NET 프레임워크와 함께 사용되며 주로 윈도우 애플리케이션과 게

In [81]:
# cache에 저장되어 있느 상태라서 바로 불러옴요
response = model.invoke("주요 프로그래밍 언어 5개 설명 부탁~")
print(response.content)

물론입니다! 아래는 현재 널리 사용되는 주요 프로그래밍 언어 다섯 가지에 대한 간단한 설명입니다.

1. **Python**:
   - **개요**: Python은 간결하고 읽기 쉬운 문법을 가지고 있어 초보자부터 전문가까지 폭넓게 사용되는 언어입니다.
   - **특징**: 다목적 언어로, 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립팅 등에 활용됩니다. 다양한 라이브러리와 프레임워크(예: Django, Flask, Pandas, TensorFlow)가 제공되어 생산성을 높입니다.

2. **JavaScript**:
   - **개요**: JavaScript는 주로 웹 페이지의 인터랙션과 동적 콘텐츠 작성을 위해 사용되는 프로그래밍 언어입니다.
   - **특징**: 클라이언트 측 언어로 시작했지만, Node.js와 같은 런타임 환경 덕분에 서버 측에서도 사용됩니다. React, Angular, Vue.js 등의 프레임워크와 함께 풍부한 웹 애플리케이션 개발을 지원합니다.

3. **Java**:
   - **개요**: Java는 객체 지향 프로그래밍 언어로, "한 번 작성하면 어디서나 실행된다"는 슬로건을 내세웁니다.
   - **특징**: 플랫폼에 독립적인 특성 덕분에 다양한 환경에서 실행 가능하며, 대규모 기업 애플리케이션과 안드로이드 앱 개발에 많이 사용됩니다. 장기적인 유지보수와 안정성을 중시하는 경우에 적합합니다.

4. **C++**:
   - **개요**: C++는 C 언어의 확장으로, 객체 지향 프로그래밍을 지원하며 시스템 프로그래밍 및 성능이 중요한 어플리케이션에 적합합니다.
   - **특징**: 게임 개발, 그래픽 소프트웨어, 임베디드 시스템 등에서 널리 사용됩니다. C++는 메모리 관리와 성능을 세밀하게 조정할 수 있어 복잡한 시스템의 개발에 강력한 도구가 됩니다.

5. **C#**:
   - **개요**: C#은 마이크로소프트가 만든 프로그래밍 언어로, .NET 프레임워크와 함께 사용되며 주로 윈도우 애플리케이션과 게

In [82]:
model.invoke("Langchain에 대해 설명 부탁 ~")

AIMessage(content='Langchain은 자연어 처리(NLP)와 관련된 다양한 작업을 쉽게 수행할 수 있도록 돕는 프레임워크입니다. 주로 대화형 AI 시스템, 챗봇 및 다양한 자연어 처리 애플리케이션의 개발을 지원합니다. Langchain은 다음과 같은 주요 기능을 제공합니다:\n\n1. **모듈화된 구성 요소**: Langchain은 다양한 모듈들을 제공하여 개발자가 필요한 기능을 조합하여 사용할 수 있게 합니다. 예를 들어, 텍스트 전처리, 모델 선택, 후처리 등의 작업을 각각의 모듈로 나누어 처리할 수 있습니다.\n\n2. **다양한 모델 지원**: Langchain은 여러 가지 언어 모델을 통합할 수 있는 기능을 제공하여, 사용자가 OpenAI의 GPT-3, Hugging Face의 Transformers 모델 등 여러 모델 중에서 선택하여 사용할 수 있도록 합니다.\n\n3. **대화 생성**: Langchain은 사용자와의 상호작용을 위한 대화 생성 기능을 강화하여, 자연스럽고 일관된 대화를 생성할 수 있게 도와줍니다.\n\n4. **사용자 정의**: 개발자가 요구하는 특정 니즈에 맞게 모델을 조정하거나 특화된 데이터를 사용하여 맞춤형 모델을 쉽게 구축할 수 있는 기능을 제공합니다.\n\n5. **데이터 통합**: Langchain은 외부 데이터베이스 및 API와의 통합을 용이하게 하여, 더 많은 데이터를 기반으로 한 강력한 기능을 구현할 수 있습니다.\n\nLangchain은 이러한 구조 덕분에 개발자들이 빠르게 프로토타입을 만들고, 실험하며, 다양한 자연어 처리 문제를 해결하는데 도움을 줍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 369, 'prompt_tokens': 14, 'total_tokens': 383, 'completion_tokens_details': {'accepted_prediction

In [83]:
model.invoke("Langchain에 대해 설명 부탁 ~")

AIMessage(content='Langchain은 자연어 처리(NLP)와 관련된 다양한 작업을 쉽게 수행할 수 있도록 돕는 프레임워크입니다. 주로 대화형 AI 시스템, 챗봇 및 다양한 자연어 처리 애플리케이션의 개발을 지원합니다. Langchain은 다음과 같은 주요 기능을 제공합니다:\n\n1. **모듈화된 구성 요소**: Langchain은 다양한 모듈들을 제공하여 개발자가 필요한 기능을 조합하여 사용할 수 있게 합니다. 예를 들어, 텍스트 전처리, 모델 선택, 후처리 등의 작업을 각각의 모듈로 나누어 처리할 수 있습니다.\n\n2. **다양한 모델 지원**: Langchain은 여러 가지 언어 모델을 통합할 수 있는 기능을 제공하여, 사용자가 OpenAI의 GPT-3, Hugging Face의 Transformers 모델 등 여러 모델 중에서 선택하여 사용할 수 있도록 합니다.\n\n3. **대화 생성**: Langchain은 사용자와의 상호작용을 위한 대화 생성 기능을 강화하여, 자연스럽고 일관된 대화를 생성할 수 있게 도와줍니다.\n\n4. **사용자 정의**: 개발자가 요구하는 특정 니즈에 맞게 모델을 조정하거나 특화된 데이터를 사용하여 맞춤형 모델을 쉽게 구축할 수 있는 기능을 제공합니다.\n\n5. **데이터 통합**: Langchain은 외부 데이터베이스 및 API와의 통합을 용이하게 하여, 더 많은 데이터를 기반으로 한 강력한 기능을 구현할 수 있습니다.\n\nLangchain은 이러한 구조 덕분에 개발자들이 빠르게 프로토타입을 만들고, 실험하며, 다양한 자연어 처리 문제를 해결하는데 도움을 줍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 369, 'prompt_tokens': 14, 'total_tokens': 383, 'completion_tokens_details': {'accepted_prediction

# streaming 방식 응답 처리 !
- LLM 응답 완료를 기다리지 않고 실시간으로 우다다 생성되는 토큰/청크를 받아서 처리함
- `모델.invoke(input, config) -> 응답 데이터` : 모델의 응답을 기달뎠다가 완료되면 한번에 반환 (마지막에 `<EOS>` 출력되면 토큰들 묶어서 줌)
- `모델.stream(input, config) -> Iterator` : 모델이 토큰을 생성하면 바로바로 반환 (timestep 마다 나오는 토큰들을 계속 줌)

In [84]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name = "gpt-4o-mini")
res = model.stream("AI에 대해서 설명 부탁 ~")
print(type(res))	# Iterator 안에 generator가 있삼

<class 'generator'>


In [85]:
for token in model.stream("AI에 대해서 설명 부탁 ~"):
    print(token.content, end="")

AI(인공지능)는 인간의 지능을 모방하거나 확장하는 컴퓨터 시스템이나 프로그램을 의미합니다. AI는 여러 분야에서 다양한 형태로 발전하고 있으며, 크게 두 가지 범주로 나눌 수 있습니다.

1. **약한 AI(Weak AI)**: 특정 작업이나 문제를 해결하기 위해 설계된 AI로, 이론적인 인공지능을 가지고 있지 않습니다. 예를 들어, 음성 인식 소프트웨어, 추천 시스템, 자율주행차의 경량 알고리즘 등이 이에 해당합니다.

2. **강한 AI(Strong AI)**: 인간과 같은 수준의 지능을 갖춘 AI로, 일반적인 문제 해결 및 학습 능력을 갖추고 있습니다. 현재로서는 강한 AI는 이론적 개념에 가깝고, 실제로 존재하지 않습니다.

AI는 머신러닝(Machine Learning)과 딥러닝(Deep Learning) 같은 기술을 통해 발전하고 있습니다. 머신러닝은 데이터에서 학습하여 패턴을 인식하고 예측하는 기술이며, 딥러닝은 인공신경망을 이용하여 더욱 복잡한 문제를 해결하는 하위 분야입니다.

AI의 응용 분야는 매우 광범위하며, 예를 들어 의료 진단, 자연어 처리, 이미지 인식, 게임, 로봇 공학 등에서 활용되고 있습니다. AI 기술은 산업 전반에 걸쳐 혁신을 이루고 있으며, 앞으로도 계속해서 발전할 것으로 예상됩니다. 

AI의 발전은 여러 가지 윤리적, 사회적 질문을 야기하고 있으며, 이에 대한 논의도 활발히 이루어지고 있습니다.